In [ ]:
#Ryan Burgert 2024

#Setup:
#    Run this in a Jupyter Notebook on a computer with at least one GPU
#        `sudo apt install ffmpeg git`
#        `pip install rp`
#    The first time you run this it might be a bit slow (it will download necessary models)
#    The `rp` package will take care of installing the rest of the python packages for you

import rp

rp.git_import('CommonSource') #If missing, installs code from https://github.com/RyannDaGreat/CommonSource
import rp.git.CommonSource.noise_warp as nw

FRAME = 2**-1 #We immediately resize the input frames by this factor, before calculating optical flow
              #The flow is calulated at (input size) × FRAME resolution.
              #Higher FLOW values result in slower optical flow calculation and higher intermediate noise resolution
              #Larger is not always better - watch the preview in Jupyter to see if it looks good!
FLOW = 2**4   #Then, we use bilinear interpolation to upscale the flow by this factor
              #We warp the noise at (input size) × FRAME × FLOW resolution
              #The noise is then downsampled back to (input size)
              #Higher FLOW values result in more temporally consistent noise warping at the cost of higher VRAM usage and slower inference time
LATENT = 8    #We further downsample the outputs by this amount - because 8 pixels wide corresponds to one latent wide in Stable Diffusion
              #The final output size is (input size) ÷ LATENT regardless of FRAME and FLOW

LATENT = 2    #Uncomment this line for a prettier visualization! But for Marigold, use LATENT=8


#Here are some Eyeline examples! Uncomment one
images = "/fsx_scanline/from_eyeline/users/ahmet/siggraph_asia_2024_videos/source_data/KW/camera_tt_360/dynamic_0007/cam_*_color.png"
# images = "/fsx_scanline/from_eyeline/users/ahmet/siggraph_asia_2024_videos/source_data/KW/three_quarters_static/dynamic_0007/cam_*_color.png"
# images = "/efs/users/ryan.burgert/public/sharing/kevin_matte_video_Jun28_2024/rgb_on_white/*png"

#You can also use video files or URLs
images = "https://www.shutterstock.com/shutterstock/videos/1100085499/preview/stock-footage-bremen-germany-october-old-style-carousel-moving-on-square-in-city-horses-on-traditional.webm"

output_folder = rp.get_folder_name(rp.get_parent_folder(images))

#See this function's docstring for more information!
output = nw.get_noise_from_video(
    images,
    remove_background=False, #Set this to True to matte the foreground - and force the background to have no flow
    save_files=True,         #Set this to False if you just want the noises without saving to a numpy file
    visualize=True,          #Generates nice visualization videos and previews in Jupyter notebook
    visualize_flow_sensitivity = .05,
    
    noise_channels=4,
    output_folder=output_folder,
    resize_frames=FRAME,
    resize_flow=FLOW,
    downscale_factor=round(FRAME * FLOW) * LATENT,
);

print("Noise shape:"  ,output.numpy_noises.shape)
print("Flow shape:"   ,output.numpy_flows .shape)
print("Output folder:",output.output_folder)